In [ ]:
from bayesbeat.data import get_n_entries
import h5py
import matplotlib.pyplot as plt
import numpy as np
import pathlib

from utils import (
    model_colours,
    model_labels,
)

plt.style.use("paper.mplstyle")

In [ ]:
outdir = pathlib.Path("figures")
outdir.mkdir(exist_ok=True)

file_format = "pdf"

Path to the data file

In [ ]:
data_file = "../data/PyTotalAnalysis_2024_02_23.mat"
n_ringdowns = get_n_entries(data_file)

In [ ]:
paths = {
    "model_1_constant_noise": pathlib.Path("../analysis/gens_data/rundir/model_1_constant_noise/"),
    "model_1": pathlib.Path("../analysis/gens_data/rundir/model_1"),
    "model_3": pathlib.Path("../analysis/gens_data/rundir/model_3_7_terms/"),
}

In [ ]:
nevals = {}
times = {}
ll_time = {}
for key, path in paths.items():
    nevals[key] = np.empty(n_ringdowns)
    times[key] = np.empty(n_ringdowns)
    ll_time[key] = np.empty(n_ringdowns)
    for index in range(n_ringdowns):
        result_file=path / "analysis" / f"index_{index}" / "result.hdf5"
        with h5py.File(result_file, "r") as res_file:
            nevals[key][index] = res_file["total_likelihood_evaluations"][()]
            times[key][index] = res_file["sampling_time"][()]
            ll_time[key][index] = res_file["likelihood_evaluation_time"][()]


In [ ]:
fig, ax = plt.subplots(1, 1)

for key in paths:

    ax.hist(
        ll_time[key] / times[key],
        color=model_colours[key],
        bins=10,
        histtype="step",
        label=model_labels[key],
    )

ax.set_yticklabels([])
ax.set_xlabel("Fractional likelihood time")

legend = ax.legend(
    frameon=True,
    fancybox=False,
    edgecolor="black",
)
legend.get_frame().set_linewidth(0.5)

plt.show()
fig.savefig(outdir / f"ll_time_fraction.{file_format}", dpi=300)


In [ ]:
fig, ax = plt.subplots(1, 1)

markers = {
    "model_1_constant_noise": "o",
    "model_1": "s",
    "model_3": "^",
}

for key in paths:
    ax.scatter(
        times[key] / 60,
        nevals[key],
        marker=markers.get(key),
        label=model_labels[key],
        color=model_colours[key],
        s=10,
    )
# ax.set_yscale("log")
ax.set_xscale("log")

ax.set_xlabel("Time [minutes]")
ax.set_ylabel("Likelihood evaluations")

legend = ax.legend(
    frameon=True,
    fancybox=False,
    edgecolor="black",
    loc="upper left",
    
)
legend.get_frame().set_linewidth(0.5)

ax.axvline(60, color="k", linestyle="--", lw=0.5)
ax.text(60, 1.5e6, "1 hour", rotation=90, ha="right", va="bottom", fontsize=8)
fig.savefig(
    outdir / f"time_vs_nevals.{file_format}", dpi=300
)
plt.show()
